In [ ]:
#############################################################################################################
## Draw least cost paths between selected villages and nearest road or village
## Coded specifically towards data being provided by researcher, HK
## Can easily be adapted to other input data. Changes would probably be need to made to the date logic
## 
## Currently configured to run cell-by-cell in a Notebook. Should be configured properly with functions and main

In [ ]:
import arcpy, os
arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "75%"

arcpy.env.extent = None

## Initialize group variable to determine which group of villages to loop through
group = 1

In [ ]:
## Input data
## Data should be copied to the same directory that this script is being stored
path = os.path.abspath(os.getcwd())

road_points_layer = path + r'\Data.gdb\nonURRAP_roads_points'   
villages_layer = path + r'\Data.gdb\kebele_centroid'
cost_raster = path + r'\Data.gdb\CostRaster'

## Copy points layers to in_memory layers to speed processing
road_points = arcpy.CopyFeatures_management(road_points_layer, r"in_memory\road_points")
villages = arcpy.CopyFeatures_management(villages_layer, r"in_memory\villages")

village_fields = [field.name for field in arcpy.ListFields(villages)]

## Remove existing paths from previous run of the tool. Comment this out if the goal is to add to existing data. 
arcpy.management.DeleteFeatures(out_paths)

In [ ]:
## function to expand extent of input features
def setExtent(in_layer, expand):
    if expand:
        x_exp = 500
        y_exp = 300
    else:
        x_exp = 0
        y_exp = 0
    desc = arcpy.Describe(in_layer)

    xmin = desc.extent.XMin - x_exp
    xmax = desc.extent.XMax + x_exp
    ymin = desc.extent.YMin - y_exp
    ymax = desc.extent.YMax + y_exp
    
    extent = arcpy.Extent(xmin, ymin, xmax, ymax)

    return extent

In [ ]:
## initialize enviroment extent to full extent of CostRaster
arcpy.env.extent = setExtent(cost_raster, True)

for cnt, village in enumerate(arcpy.da.SearchCursor(villages, field_names='*', where_clause = "GROUP_ = " + group), 1):
## reset processing extent to full extent of CostRaster for each iteration
    arcpy.env.extent = setExtent(cost_raster, True)

## Select next village and create in_memory layer for merge
    village_obj_id = village[village_fields.index('OBJECTID_1')]
    where = "OBJECTID_1 = " + str(village_obj_id)    
    village_point = arcpy.management.MakeFeatureLayer(villages, where_clause=where)

    print_str = where   

## Select nearest road point and create in_memory layer for merge    
    rdpoint_obj_id = village[village_fields.index('NEAR_FID')]
    where = "OBJECTID = " + str(rdpoint_obj_id)
    road_point = arcpy.management.MakeFeatureLayer(road_points, where_clause=where)    
    
    print_str += " and " + where
    print(print_str)
    
## Merge above layers into a single merge layer for using in the Cost Connectivity tool     
    merge_layer = arcpy.management.Merge([road_point,village_point], r"in_memory\merge_layer", "NO_SOURCE_INFO")

## Merge layer is overwritten with each iteration. Recalculate the extent for the new points.
    arcpy.management.RecalculateFeatureClassExtent(merge_layer)
    
## Limit processing extent to expanded extent of the two points. This helps decrease processing time       
    arcpy.env.extent = setExtent(merge_layer, True)
    
    try:
## Create least cost path between merged points        
        out_path = arcpy.sa.CostConnectivity(merge_layer, cost_raster, r"in_memory\out_path", None)

## Create update cursor for feature class 
        with arcpy.da.UpdateCursor(out_path, update_fields) as cursor:
            for row in cursor:
                row[0] = village_obj_id
                row[1] = rdpoint_obj_id
                cursor.updateRow(row)
## If Cost Connectivity fails, create a straight line between points.                
    except Exception as err:
        try:
            update_fields = ['REGION1', 'REGION2', 'PATHCOST']
            out_path = arcpy.management.PointsToLine(merge_layer, r"in_memory\merge_layer_PointsToLine", None, "OID", "NO_CLOSE")        
            arcpy.AddField_management(out_path,'REGION1', "LONG")
            arcpy.AddField_management(out_path,'REGION2', "LONG")
            arcpy.AddField_management(out_path,'PATHCOST', "DOUBLE")

            with arcpy.da.UpdateCursor(out_path, update_fields) as cursor:
                for row in cursor:
                    row[0] = village_obj_id
                    row[1] = rdpoint_obj_id
                    row[2] = 0
                    cursor.updateRow(row)                
            print("straight line") 
        except Exception as err:
            print("ERROR:",str(err))
            continue

## Append least cost path to paths layer
    try:
        arcpy.management.Append(out_path, out_paths, schema_type="NO_TEST")
    except Exception as err:
        print("Append ERROR:",str(err))